In [ ]:
!pip install tekore

In [ ]:
%%writefile authorization_arch.py
import tekore as tk

def authorize():
  client_id  = ""
  secret_key = ""
  token = tk.request_client_token(client_id, secret_key)
  return tk.Spotify(token)

Writing authorization_sourav.py


# Valence-Arousal-Recommendation System

In this notebook, we use a dataset featuring around 12,000 Spotify tracks, each with a `valence` and an `energy` value, to build a simple mood-based recommendation system.

In [ ]:
import pandas as pd
import random
import authorization_arch
import numpy as np
from numpy.linalg import norm

## 1. Preparations

In [ ]:
df = pd.read_csv("song_data.csv")
print(df.shape)
df.head()

(513, 20)


,SL,playlistID,TrackName,TrackID,SampleURL,ReleaseYear,Genres,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key,mode,duration_ms,Popularity
0,0,4kpm50ItxF6iFUATTyQdOR,"Jeena Jeena (From ""Badlapur"")",6h09y5UH6sDRERJALlaJPi,https://p.scdn.co/mp3-preview/804fd0793a7a06c2...,2015-01-15,"['classic pakistani pop', 'desi pop', 'filmi',...",0.676,0.319,-9.201,0.0279,0.78300,0.000000,0.1170,0.565,105.002,5,1,228969,67.0
1,1,4kpm50ItxF6iFUATTyQdOR,Samjhawan,0rk2X5TAhraBC5aCIXK2Rq,https://p.scdn.co/mp3-preview/c1f10ab622da6ca9...,2014-06-19,"['classic pakistani pop', 'pakistani pop']",0.651,0.482,-8.666,0.0281,0.29300,0.000000,0.0811,0.517,106.993,4,1,269185,64.0
2,2,4kpm50ItxF6iFUATTyQdOR,"Tum Hi Ho Bandhu (From ""Cocktail"")",4pE05HNzmVFGpCOfNKE3w6,https://p.scdn.co/mp3-preview/19b13d9981089cc6...,2012-06-12,"['desi hip hop', 'desi pop', 'filmi', 'modern ...",0.714,0.892,-5.342,0.0516,0.00145,0.000817,0.0462,0.899,134.997,11,0,282012,66.0
3,3,4kpm50ItxF6iFUATTyQdOR,"Dooriyan (From ""Love Aaj Kal"")",4peiaLdhDgUgQ646Gpbvh3,https://p.scdn.co/mp3-preview/eba63b1b8a2605a1...,2009,"['desi pop', 'filmi', 'modern bollywood']",0.517,0.716,-6.582,0.0509,0.61100,0.000000,0.1100,0.594,122.952,7,0,337360,63.0
4,4,4kpm50ItxF6iFUATTyQdOR,"Galliyan (From ""Ek Villain"")",517bz7zkWELmyL4eZzJ5R9,https://p.scdn.co/mp3-preview/47e95f698a0a66d4...,2017-03-03,"['desi pop', 'filmi', 'modern bollywood']",0.552,0.750,-5.215,0.0390,0.44500,0.000074,0.2910,0.400,89.999,10,0,340400,62.0


__Load Data__

In [29]:
df = pd.read_csv("song_data")
print(df.shape)
df.head()

(586672, 20)


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In order to compute distances between two tracks, we need to transform the seperate `valence`and `energy` columns to a `mood-vector` column.
This can be done by using `df.apply()` alongside a lambda function.

__Create Mood Vector__

In [30]:
df["mood_vec"] = df[["valence", "energy"]].values.tolist()
df["mood_vec"].head()

0     [0.127, 0.445]
1     [0.655, 0.263]
2     [0.457, 0.177]
3    [0.397, 0.0946]
4     [0.196, 0.158]
Name: mood_vec, dtype: object

__Authorize Spotify API Access__

In [31]:
sp = authorization_arch.authorize() # Use the authorization script provided earlier in the blog post

## 2. Recommendation Algorithm

The algorithm that finds similar tracks to a given input track is now very simple. 
1. Crawl the track's `valence` and `energy` values from the Spotify API.
2. Compute the distances of the input track to each track in the reference dataset.
3. Sort the reference track from lowest to highest distance.
4. Return the `n` most similar tracks.

In [35]:
def recommend(track_id, ref_df, sp, n_recs = 5):
    # Crawl valence and arousal of given track from spotify api
    track_features = sp.track_audio_features(track_id)
    track_moodvec = np.array([track_features.valence, track_features.energy])
    print(f"mood_vec for {track_id}: {track_moodvec}")
    
    # Compute distances to all reference tracks
    ref_df["distances"] = ref_df["mood_vec"].apply(lambda x: norm(track_moodvec-np.array(x)))
    # Sort distances from lowest to highest
    ref_df_sorted = ref_df.sort_values(by = "distances", ascending = True)
    # If the input track is in the reference set, it will have a distance of 0, but should not be recommendet
    ref_df_sorted = ref_df_sorted[ref_df_sorted["id"] != track_id]
    # Return n recommendations
    return ref_df_sorted.iloc[:n_recs]

Let us try it out using some random tracks from our dataset.

In [36]:
track1 = random.choice(df['id'])
recommend(track_id = track1, ref_df = df, sp = sp, n_recs = 5)

mood_vec for 1ZQoMVoisNR2UzZ54hWmnl: [0.546 0.623]


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,mood_vec,distances
560620,0VUJJCQrbEKAWZFUH7FAA7,Little Mermaid (Live Version),2,309030,0,['T-SQUARE'],['7BwOjwl5mKpGVIvzvqEcie'],2016-01-20,0.550,0.623,9,-11.015,1,0.0396,0.0411,0.063300,0.746,0.546,117.506,4,"[0.546, 0.623]",0.000
156245,6NkGUkxzYVUxr1SBXq2iuM,Stay,7,210400,0,"['Zedd', 'Alessia Cara']","['2qxJFvFYMEDqd7ui6kSAcq', '2wUjUUtkb5lvLKcGKs...",2020-12-16,0.689,0.623,5,-5.025,0,0.0621,0.2540,0.000000,0.113,0.546,101.998,4,"[0.546, 0.623]",0.000
110198,1FVv3XLNjEeDFevssQ18zz,Stay,9,210400,0,"['Zedd', 'Alessia Cara']","['2qxJFvFYMEDqd7ui6kSAcq', '2wUjUUtkb5lvLKcGKs...",2020-09-25,0.689,0.623,5,-5.025,0,0.0621,0.2540,0.000000,0.113,0.546,101.998,4,"[0.546, 0.623]",0.000
447210,2U184iOGIXks5ptjB7VEGS,Mengapa Tak Pernah Jujur,17,277200,0,['Dian Piesesha'],['1VXrpcuRNWPy5Ev4PZTKlY'],1991-07-01,0.536,0.623,5,-3.864,1,0.0304,0.4380,0.000004,0.465,0.546,130.292,4,"[0.546, 0.623]",0.000
207566,6SXBWHbor9GYbAXqnXWsG4,Alza Tu Copa y Brindemos Por Ella,24,181933,0,['El Greco'],['0Ufn5wctWw9TWhFUhb43Np'],1995-03-07,0.440,0.623,7,-9.980,0,0.0416,0.8630,0.000016,0.163,0.547,123.118,4,"[0.547, 0.623]",0.001


In [39]:
Tum_Hi_Ho_Bandhu = "4pE05HNzmVFGpCOfNKE3w6"
recommend(track_id = Tum_Hi_Ho_Bandhu , ref_df = df, sp = sp, n_recs = 10)

mood_vec for 4pE05HNzmVFGpCOfNKE3w6: [0.899 0.892]


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,mood_vec,distances
540166,2zZTXtLqCsICzEcztKjn47,"Tum Hi Ho Bandhu (From ""Cocktail"")",52,282012,0,"['Neeraj Shridhar', 'Kavita Seth']","['3tHD07u1ON4uHxmnT9rwqZ', '3nQ125TJobosBH446D...",2012-06-12,0.714,0.892,11,-5.342,0,0.0516,0.00145,0.000817,0.0462,0.899,134.997,4,"[0.899, 0.892]",0.000000
342222,5p6TThhdr9Z5BsUwQoFUIR,One More Day,32,179867,0,['Clay Davidson'],['6F0ZVOJymgjutBNRws94qa'],2000-01-01,0.573,0.892,7,-6.579,1,0.0298,0.17700,0.000171,0.4240,0.898,113.043,4,"[0.898, 0.892]",0.001000
71016,2KZXnN63ae6G6Cai2NhPkG,The Shake,52,213333,0,['Neal McCoy'],['69Etpls5a2aATUkB6NInTY'],1997-10-17,0.677,0.892,9,-5.709,1,0.0527,0.07550,0.000007,0.2020,0.898,145.921,4,"[0.898, 0.892]",0.001000
584107,1gHIx36l78G665439IGr56,Hon finns överallt,43,189058,0,['Stiftelsen'],['00PLVqZ4oaOmbI4lacLUBO'],2015-03-25,0.568,0.893,0,-4.221,1,0.0293,0.05150,0.000003,0.1410,0.900,167.070,4,"[0.9, 0.893]",0.001414
334398,4gfNIjXLL33gekozQ1lIwW,Rubi,46,208200,0,"['Banda Djavú', 'DJ Juninho Portugal']","['4adpoL1QBRZW6jWjRVMB04', '0NFJAW3QGhPDdZAuIM...",2009-02-02,0.730,0.893,10,-2.940,0,0.0471,0.32500,0.000025,0.2800,0.900,170.042,4,"[0.9, 0.893]",0.001414
580383,799bo1xWDF9jFdNzLmXUze,Liven,19,156173,0,['Wild Rover'],['57ggG9OTUwTfffR0H9ORAq'],1991,0.455,0.891,7,-8.904,1,0.0578,0.22800,0.000000,0.3910,0.900,159.207,4,"[0.9, 0.8909999999999999]",0.001414
100380,00DJt4PjkzeXhKKVDekw2n,Music Man,9,220133,0,['Letta Mbulu'],['7iwwcDXXToUdUoDYP70EA0'],1976-01-01,0.770,0.891,1,-7.306,1,0.1720,0.54300,0.000796,0.0684,0.898,135.573,4,"[0.898, 0.8909999999999999]",0.001414
259171,1NwBkpyg2jNCaA9lRyiBUL,Slnečný kalendár,18,162187,0,['Marika Gombitova'],['3Tn9kQpK0Yeh3Dk0DhND4m'],1982-01-01,0.509,0.891,7,-5.348,1,0.0361,0.06940,0.220000,0.1200,0.900,91.445,4,"[0.9, 0.8909999999999999]",0.001414
215763,2voMLLjHUMn6fcSG0FmHDC,Que Te Aleja de Mi,29,276349,0,['Galy Galiano'],['2OSKh2JIKMr66DxByxhwzT'],1993-03-30,0.683,0.892,0,-6.147,1,0.0350,0.69900,0.004490,0.1490,0.901,168.423,4,"[0.9009999999999999, 0.892]",0.002000
293985,6YuUwHREUIEEMOjoqMs7vP,Party,27,216000,0,"['Monsieur Minimal', 'Marietta Fafouti']","['530wyaQMvivGiC0P00MUml', '6Q2VEtumYQdSbszI6A...",2012-11-26,0.548,0.892,0,-3.162,1,0.0257,0.26200,0.001020,0.4050,0.901,90.001,4,"[0.9009999999999999, 0.892]",0.002000


In [ ]:
rosanna = "2n9ylYiF7urpXo36qBwr3g"
recommend(track_id = rosanna, ref_df = df, sp = sp, n_recs = 5)

mood_vec for 2n9ylYiF7urpXo36qBwr3g: [0.375 0.733]


,SL,playlistID,TrackName,TrackID,SampleURL,ReleaseYear,Genres,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key,mode,duration_ms,Popularity,mood_vec,distances
327,327,4kpm50ItxF6iFUATTyQdOR,Mitti Di Khushboo,7tp4iYvcZksGp1u7feUrrO,https://p.scdn.co/mp3-preview/54426f2f3e3f06fa...,2014-10-08,"['desi pop', 'modern bollywood']",0.640,0.754,-4.873,0.0392,0.395,0.000000,0.209,0.355,94.989,6,0,247223,59.0,"[0.355, 0.754]",0.029000
4,4,4kpm50ItxF6iFUATTyQdOR,"Galliyan (From ""Ek Villain"")",517bz7zkWELmyL4eZzJ5R9,https://p.scdn.co/mp3-preview/47e95f698a0a66d4...,2017-03-03,"['desi pop', 'filmi', 'modern bollywood']",0.552,0.750,-5.215,0.0390,0.445,0.000074,0.291,0.400,89.999,10,0,340400,62.0,"[0.4, 0.75]",0.030232
7,7,4kpm50ItxF6iFUATTyQdOR,Maula Mere Maula,4CGBTtlrjZj7ydpV52cgB4,https://p.scdn.co/mp3-preview/3161cb1eef34ef20...,2007-01-12,"['classic bollywood', 'desi pop', 'filmi', 'mo...",0.549,0.737,-7.865,0.0386,0.459,0.008440,0.091,0.412,119.963,0,1,364330,64.0,"[0.41200000000000003, 0.737]",0.037216
70,70,4kpm50ItxF6iFUATTyQdOR,Be Intehaan,6g0HHWLJwfe2PlbgHi8pBj,https://p.scdn.co/mp3-preview/524480bc471eafbf...,2012-01-25,"['classic pakistani pop', 'desi pop', 'filmi',...",0.560,0.696,-5.583,0.0473,0.535,0.000000,0.325,0.390,128.849,7,1,290867,63.0,"[0.39, 0.696]",0.039925
415,415,4kpm50ItxF6iFUATTyQdOR,Aadat,6KUlOsfQ6N72BIu1Yl1q9y,https://p.scdn.co/mp3-preview/58fa1e1935fe857c...,2005-12-09,"['classic pakistani pop', 'desi pop', 'filmi',...",0.622,0.746,-6.551,0.0426,0.175,0.000872,0.271,0.336,102.962,0,0,333740,48.0,"[0.336, 0.746]",0.041110
